In [ ]:
if __name__ == '__main__':         
    #! REGION ============== Start CHange =================
    fileFitur = 'faceRecognotionOK3.npz'
    path = "lfwTest" 
    nmFileAug = "faceRecognotionAug3.npz"
    if os.path.exists(nmFileAug): 
        dataLoaded = np.load(nmFileAug)       
        x_train  =dataLoaded['x_train']
        y_train  =dataLoaded['y_train']
        x_val    =dataLoaded['x_val']
        y_val    =dataLoaded['y_val']
        num_classes=dataLoaded['num_classes']
        print("Data loaded ...")
    else:
        detector = mtcnn.MTCNN()        
        if os.path.exists(fileFitur): 
           print("File %s already exist"%fileFitur)           
           dataLoaded = np.load(fileFitur)       
           ftr  =dataLoaded['ftr']
           nm   =dataLoaded['nm']
           print("loaded success")  
        else:              
            ftrs,nms = [],[],[],[]
            
            for i in range(len(train)):
                nmPath = path
 
                ftr, nm = loadImageDetectFace(nmPath) 
                numFaceDetected = len(list(set(nm)))
                if numFaceDetected !=3:
                     print("tidak lengkap 3 face dalam 1 folder %d, try with fr"%numFaceDetected)
                     ftr, nm = loadImageDetectFace(nmPath,fr=True)     
                     numFaceDetected = len(list(set(nm)))
                     if numFaceDetected !=3:
                         print("tidak lengkap 3 face dalam 1 folder %d "%(numFaceDetected))
                
                if numFaceDetected ==3:
                     ftrs+=ftr
                     nms+=nm
             
                print("############   %d"%i)
        
            np.savez_compressed(fileFitur,ftr=np.array(ftrs),
                              # lbl=np.array(lbls),
                              nm=np.array(nms),
                              # usia=np.array(usias))
                              )
         
        ftr = getResize(ftr)
        ftr=np.asarray(ftr).astype(np.float32)    
        #! END REGION ============== End CHange =================
        # TODO: print ftr
        
        print("splitting traing and validation")
        print("-------------------------------")
   
    



In [ ]:
def loadImageDetectFace(path, fr=False):    
    ftr,nmFiles,idImgs,scoreFace,idNumber  =[],[],[],[],0    
    
    #! REGION FACE DETECTION
    for nmFile in os.listdir(path):
        print("nmFile ...." + nmFile)
        image_path = path+"/"+nmFile
        extracted_face = extract_face_from_image(image_path, 
                                                 required_size=(224, 224))                
        if len(extracted_face)>0:
            if fr==True:
                for detected_face in extracted_face:
                    fe = getEncoding(detected_face)
                    if len(fe)>0:
                        scoreFace.append(fe)
                        ftr.append(detected_face)                      
                        nmFiles.append(nmFile)      
                        idImgs.append(idNumber)
                        idNumber+=1  
            else:
                model_scores   = get_model_scores(extracted_face)        
                for i in range(len(model_scores)):
                    scoreFace.append(model_scores[i])
                    ftr.append(extracted_face[i])                      
                    nmFiles.append(nmFile)      
                    idImgs.append(idNumber)
                    idNumber+=1
                    
               
    comb      = combinations(idImgs, 2)
    similars  = np.zeros(len(idImgs))
    for c in comb:
        # print(c)
        matches=False
        if fr==True :
            matches = face_recognition.compare_faces(np.array(scoreFace[c[0]]), 
                                                     np.array(scoreFace[c[1]]))

        else:
            diff=cosine(scoreFace[c[0]], scoreFace[c[1]])
            print(diff)
            if  diff <= 0.4:
                matches=True
            
        if matches:
            print("Faces Matched")                            
            similars[c[0]]+=1
            similars[c[1]]+=1
        else:
            print("ada yg tidak match")
            plt.imshow(ftr[c[0]])
            plt.show()      
            plt.imshow(ftr[c[1]])
            plt.show()      

    df = pd.DataFrame({'id':np.array(idImgs), 'mirip':similars})          
    df = df.sort_values(by=['mirip'],ascending=False)  
    df = df[df['mirip']>1]
    df.reset_index(drop=True, inplace=True)
    #get base on 3 folder->3 face sj
    # selectedFtr,selectedLbl,selectedUsia,selectNm=[],[],[],[]
    selectedFtr,selectNm=[],[]
    cntFace=0
    for d in range(len(df)):  
        #! INI FEATURE NYA
        #TODO: pelajarin apa yang ada di idImg ini
        idImg = df['id'].loc[d]        
        if nmFiles[idImg] not in selectNm:
            selectedFtr.append(ftr[idImg])
            # selectedLbl.append(jk)               
            # selectedUsia.append(umur)
            selectNm.append(nmFiles[idImg])
            cntFace+=1
        if cntFace >= 3:
            break            
            
    # return selectedFtr,selectedLbl,selectedUsia,selectNm   
    return selectedFtr,selectNm   
